In [1]:
cd ..

C:\Users\OLOYEDE_O\PycharmProjects\scraper


In [34]:
from scraper.jobs.utils import get_driver
from scraper.jobs.utils import convert_bbl_to_kbd
import pandas as pd
import selenium
from selenium import webdriver
import tabula
import requests
import glob
import pathlib

In [35]:
#specify the page link
page_url = 'https://www.petrocom.gov.gh/archive/'

In [4]:
# create the selenium object
driver = get_driver(headless=True)

In [5]:
# create the selenium object
driver.get(page_url)

In [6]:
file_link = []

In [7]:
ten_field = driver.find_elements_by_xpath('//table/tbody/tr[2]/td[3]/a')
jubilee_field = driver.find_elements_by_xpath('//table/tbody/tr[3]/td[3]/em[1]/a')
oct_field = driver.find_elements_by_xpath('//table/tbody/tr[4]/td[3]/a')

In [8]:
for link in ten_field:
    file_link.append(link.get_attribute('href'))

In [9]:
for link in jubilee_field:
    file_link.append(link.get_attribute('href'))

In [10]:
for link in oct_field:
    file_link.append(link.get_attribute('href'))

In [11]:
file_link

['https://www.petrocom.gov.gh/wp-content/uploads/2021/03/ten-prod.pdf',
 'https://www.petrocom.gov.gh/wp-content/uploads/2020/02/2019-TEN-Production.pdf',
 'https://www.petrocom.gov.gh/wp-content/uploads/2019/05/2018-Ten-Productions.pdf',
 'https://www.petrocom.gov.gh/wp-content/uploads/2021/03/jubilee-prod.pdf',
 'https://www.petrocom.gov.gh/wp-content/uploads/2020/02/2019-Jubilee-Production.pdf',
 'https://www.petrocom.gov.gh/wp-content/uploads/2019/05/2018-Jubilee-Productions.pdf',
 'https://www.petrocom.gov.gh/wp-content/uploads/2021/03/octp-prod.pdf',
 'https://www.petrocom.gov.gh/wp-content/uploads/2020/02/2019-OCTP-SANKOFA-GYE-NYAME-PRODUCTION.pdf',
 'https://www.petrocom.gov.gh/wp-content/uploads/2019/05/2018-OCTP-Productions.pdf']

In [111]:
def ghana_historicals(file_link):
    """
    This method takes in the position of the file and returns the final processed dataframe
    @ returns a dataframe
    """
    # read in the 
    df = tabula.read_pdf(file_link,pages = 1,user_agent='chrome',output_format ='dataframe',lattice=True)[0]
    # remove special character
    df= df.replace('\*','',regex=True)
    df= df.replace(',','',regex=True)
    # get the field name
    temp_field_name = df.columns.values[0].upper().split('PRODUCTION')[0].split()
    # rename the columns
    df.columns = df.iloc[0].values
    # reshape the dataframe
    df = df.iloc[1:,:]
    # drop columns we dont need 
    df.drop(columns=df.columns[3:],axis=1,inplace=True)
    # rename the columns
    df.rename({df.columns[1]:'oil_production',df.columns[2]:'gas_production'},axis=1,inplace=True)
    # extract the production year
    prod_year = pd.to_datetime(df['Period'][1]).year
    # change the period column
    df['Period'] = df['Period'].str.strip().str[:3].str.upper() + str(prod_year)
    # assign the field name to the dataframe
    if temp_field_name[0].isdigit():
        df['entity'] = temp_field_name[1].upper()
    else:
        df['entity'] = temp_field_name[0].upper()
        
    if df['entity'].all() =='OCTP':
        df['entity'] = 'OCTP_SANKOFA_GYENYAME'       
    # create a temporary column for the conversion of the volume 
    df['temporary_date'] = pd.to_datetime(df['Period'])
    df['oil_production'] = df['oil_production'].astype(float)
    # convert the oil volume from bbl to KBD
    df['oil_production'] = df.apply(lambda x: convert_bbl_to_kbd(x['oil_production'], x['temporary_date'].year, x['temporary_date'].month),axis = 1)
    # melt the dataframe to the required format
    df = pd.melt(df, id_vars=['entity','Period'],value_vars=['oil_production', 'gas_production'], value_name='value',var_name='product')
    # create a product mapping
    product_mapping = {'oil_production':'CRUDEOIL','gas_production':'GAS'}
    # create a product column based on whether is oil or gas
    df['product'] = df['product'].map(product_mapping)
    # assign the units based on the product type
    df['unit'] = df['product'].apply(lambda x: 'KBD' if (x == 'CRUDEOIL') else 'MMSCF')
    
    return df 

In [112]:
# create q list holding all the dataframes
final_df = []

#iterate over the file links and generate the dataframes
for link in file_link:
    df = ghana_historicals(link)
    final_df.append(df)

In [113]:
    
ghana_df = pd.concat(final_df)

In [116]:
ghana_df.to_csv('gh_oil_supply.csv',sep=';',index=False)

In [117]:
ghana_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216 entries, 0 to 23
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   entity   216 non-null    object
 1   Period   216 non-null    object
 2   product  216 non-null    object
 3   value    216 non-null    object
 4   unit     216 non-null    object
dtypes: object(5)
memory usage: 10.1+ KB


In [1]:
cd ..

C:\Users\OLOYEDE_O\PycharmProjects\scraper


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

In [4]:
# from scraper.jobs.ng_gov_dpr.nigerian_oil_supply import NigerianOilSupplyJob

from scraper.core import factory

# job = NigerianOilSupplyJob()
# job = factory.get_scraper_job('ng_gov_dpr', 'nigerian_oil_supply')

# job = NigerianOilSupplyJob(full_load=True)
job = factory.get_scraper_job('gh_gov_petrocom', 'ghana_oil_supply', full_load=True)

2021-10-27 17:11:00,100 - scraper.core.factory - DEBUG - Loading module scraper.jobs.gh_gov_petrocom.ghana_oil_supply
2021-10-27 17:11:00,159 - scraper.core.factory - DEBUG - Getting class GhanaOilSupplyJob
2021-10-27 17:11:00,709 - selenium.webdriver.remote.remote_connection - DEBUG - POST http://127.0.0.1:61857/session {"capabilities": {"firstMatch": [{}], "alwaysMatch": {"browserName": "chrome", "platformName": "any", "goog:chromeOptions": {"prefs": {"download.default_directory": "C:\\Users\\OLOYEDE_O\\PycharmProjects\\scraper\\filestore"}, "extensions": [], "args": ["--headless", "--disable-dev-shm-usage", "window-size=1920x1480"]}}}, "desiredCapabilities": {"browserName": "chrome", "version": "", "platform": "ANY", "goog:chromeOptions": {"prefs": {"download.default_directory": "C:\\Users\\OLOYEDE_O\\PycharmProjects\\scraper\\filestore"}, "extensions": [], "args": ["--headless", "--disable-dev-shm-usage", "window-size=1920x1480"]}}}
2021-10-27 17:11:00,711 - urllib3.connectionpool 

In [5]:
job.run()

2021-10-27 17:11:14,320 - scraper.jobs.gh_gov_petrocom.ghana_oil_supply - INFO - getting the sources to the tables
2021-10-27 17:11:14,321 - scraper.jobs.gh_gov_petrocom.ghana_oil_supply - DEBUG - parsing the main page to get table links at : https://www.petrocom.gov.gh/production-figures
2021-10-27 17:11:14,321 - selenium.webdriver.remote.remote_connection - DEBUG - POST http://127.0.0.1:61857/session/a3885bc19c3c1d6c3402fde26cb7762d/url {"url": "https://www.petrocom.gov.gh/production-figures"}
2021-10-27 17:11:19,111 - urllib3.connectionpool - DEBUG - http://127.0.0.1:61857 "POST /session/a3885bc19c3c1d6c3402fde26cb7762d/url HTTP/1.1" 200 14
2021-10-27 17:11:19,112 - selenium.webdriver.remote.remote_connection - DEBUG - Finished Request
2021-10-27 17:11:19,112 - scraper.jobs.gh_gov_petrocom.ghana_oil_supply - INFO - successfully retrieved data from : https://www.petrocom.gov.gh/production-figures
2021-10-27 17:11:19,114 - selenium.webdriver.remote.remote_connection - DEBUG - POST htt

OSError: Issue for loop 2: 
 b'{"message": "Internal Server Error"}\n'

In [12]:
job.get_sources()

2021-10-27 16:42:09,987 - scraper.jobs.gh_gov_petrocom.ghana_oil_supply - INFO - getting the sources to the tables
2021-10-27 16:42:09,987 - scraper.jobs.gh_gov_petrocom.ghana_oil_supply - INFO - getting the sources to the tables
2021-10-27 16:42:09,988 - scraper.jobs.gh_gov_petrocom.ghana_oil_supply - DEBUG - parsing the main page to get table links at : https://www.petrocom.gov.gh/production-figures
2021-10-27 16:42:09,988 - scraper.jobs.gh_gov_petrocom.ghana_oil_supply - DEBUG - parsing the main page to get table links at : https://www.petrocom.gov.gh/production-figures
2021-10-27 16:42:09,989 - selenium.webdriver.remote.remote_connection - DEBUG - POST http://127.0.0.1:52630/session/e720abc66f5a53deea954ed3188f1020/url {"url": "https://www.petrocom.gov.gh/production-figures"}
2021-10-27 16:42:09,989 - selenium.webdriver.remote.remote_connection - DEBUG - POST http://127.0.0.1:52630/session/e720abc66f5a53deea954ed3188f1020/url {"url": "https://www.petrocom.gov.gh/production-figures"

In [6]:
[vars(s) for s in job.sources]

[{'code': 'gh_ten_2021',
  'url': 'https://www.petrocom.gov.gh/production-figures',
  'path': 'gh_ten_2021.html',
  'long_name': 'Ghana Monthly Oil Production for 2021-TEN',
  'meta_data': {'table_id': 'tablepress-11'}},
 {'code': 'gh_jubilee_2021',
  'url': 'https://www.petrocom.gov.gh/production-figures',
  'path': 'gh_jubilee_2021.html',
  'long_name': 'Ghana Monthly Oil Production for 2021-Jubilee',
  'meta_data': {'table_id': 'tablepress-10'}},
 {'code': 'gh_octp_sankofa-gyenyame_2021',
  'url': 'https://www.petrocom.gov.gh/production-figures',
  'path': 'gh_octp_sankofa-gyenyame_2021.html',
  'long_name': 'Ghana Monthly Oil Production for 2021-OCTP_Sankofa-GyeNyame',
  'meta_data': {'table_id': 'tablepress-12'}},
 {'code': 'gh_oil_supply_2018_to_2020',
  'url': 'https://www.petrocom.gov.gh/archive/',
  'path': 'gh_oil_supply_2018_to_2020.csv',
  'long_name': 'Ghana Monthly Oil Production for 2018 to 2020'}]

In [15]:
ruue[0]['meta_data']

{'table_id': 'tablepress-11'}

In [23]:
job.transform()

2021-10-27 16:51:32,275 - selenium.webdriver.remote.remote_connection - DEBUG - GET http://127.0.0.1:52630/session/e720abc66f5a53deea954ed3188f1020/source {}
2021-10-27 16:51:32,275 - selenium.webdriver.remote.remote_connection - DEBUG - GET http://127.0.0.1:52630/session/e720abc66f5a53deea954ed3188f1020/source {}
2021-10-27 16:51:32,286 - urllib3.connectionpool - DEBUG - http://127.0.0.1:52630 "GET /session/e720abc66f5a53deea954ed3188f1020/source HTTP/1.1" 200 89931
2021-10-27 16:51:32,286 - urllib3.connectionpool - DEBUG - http://127.0.0.1:52630 "GET /session/e720abc66f5a53deea954ed3188f1020/source HTTP/1.1" 200 89931
2021-10-27 16:51:32,289 - selenium.webdriver.remote.remote_connection - DEBUG - Finished Request
2021-10-27 16:51:32,289 - selenium.webdriver.remote.remote_connection - DEBUG - Finished Request
2021-10-27 16:51:32,314 - scraper.jobs.gh_gov_petrocom.ghana_oil_supply - DEBUG - the temporary column has been dropped
2021-10-27 16:51:32,314 - scraper.jobs.gh_gov_petrocom.gha

In [28]:
job.data[150:200]

,entity,period,product,value,unit,provider,frequency,area,flow,original,source
108,JUBILEE,JAN2019,GAS,3478.47,MMSCF,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,gh_oil_supply_2018_to_2020
109,JUBILEE,FEB2019,GAS,2429.4,MMSCF,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,gh_oil_supply_2018_to_2020
110,JUBILEE,MAR2019,GAS,4317.11,MMSCF,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,gh_oil_supply_2018_to_2020
111,JUBILEE,APR2019,GAS,4413.05,MMSCF,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,gh_oil_supply_2018_to_2020
112,JUBILEE,MAY2019,GAS,4828.65,MMSCF,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,gh_oil_supply_2018_to_2020
113,JUBILEE,JUN2019,GAS,4482.45,MMSCF,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,gh_oil_supply_2018_to_2020
114,JUBILEE,JUL2019,GAS,4565.4,MMSCF,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,gh_oil_supply_2018_to_2020
115,JUBILEE,AUG2019,GAS,4844.83,MMSCF,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,gh_oil_supply_2018_to_2020
116,JUBILEE,SEP2019,GAS,4579.96,MMSCF,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,gh_oil_supply_2018_to_2020
117,JUBILEE,OCT2019,GAS,4875.38,MMSCF,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,gh_oil_supply_2018_to_2020


In [26]:
job.data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 258 entries, 0 to 215
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   entity     258 non-null    object
 1   period     258 non-null    object
 2   product    258 non-null    object
 3   value      258 non-null    object
 4   unit       258 non-null    object
 5   provider   258 non-null    object
 6   frequency  258 non-null    object
 7   area       258 non-null    object
 8   flow       258 non-null    object
 9   original   258 non-null    bool  
 10  source     258 non-null    object
dtypes: bool(1), object(10)
memory usage: 22.4+ KB
